<a href="https://www.kaggle.com/code/kapusharinka/network-anomaly-detection?scriptVersionId=147585802" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Network Anomaly Detection
https://en.wikipedia.org/wiki/Anomaly_detection#Popular_techniques</br>
https://medium.com/@AmyGrabNGoInfo/list/imbalanced-classification-and-anomalies-detection-dc908de4382d</br>
https://youtube.com/playlist?list=PLVppujud2yJo0qnXjWVAa8h7fxbFJHtfJ&feature=shared

## Data preprocessing and overview

In [ ]:
train = pd.read_csv('/kaggle/input/network-intrusion-detection/Train_data.csv')
test = pd.read_csv('/kaggle/input/network-intrusion-detection/Test_data.csv')
train.info()

### List of columns

#### BASIC FEATURES OF EACH NETWORK CONNECTION VECTOR
1. Duration: Length of time duration of the connection
1. Protocol_type: Protocol used in the connection
1. Service: Destination network service used
1. Flag: Status of the connection – Normal or Error
1. Src_bytes: Number of data bytes transferred from source to destination in single connection
1. Dst_bytes: Number of data bytes transferred from destination to source in single connection
1. Land: if source and destination IP addresses and port numbers are equal then, this variable takes value 1 else 0
1. Wrong_fragment: Total number of wrong fragments in this connection
1. Urgent: Number of urgent packets in this connection. Urgent packets are packets with the urgent bit activated
#### CONTENT RELATED FEATURES OF EACH NETWORK CONNECTION VECTOR
1. Hot: Number of „hot‟ indicators in the content such as: entering a system directory, creating programs and executing programs
1. Num_failed_logins: Count of failed login attempts
1. Logged_in Login Status: 1 if successfully logged in; 0 otherwise
1. Num_compromised: Number of compromised' ' conditions 
1. Root_shell: 1 if root shell is obtained; 0 otherwise 
1. Su_attempted: 1 ifsu root'' command attempted or used; 0 otherwise
1. Num_root: Number of root'' accesses or number of operations performed as a root in the connection 
1. Num_file_creations: Number of file creation operations in the connection 
1. Num_shells: Number of shell prompts 
1. Num_access_files: Number of operations on access control files 
1. Num_outbound_cmds: Number of outbound commands in an ftp session 
1. Is_hot_login: 1 if the login belongs to thehot'' list i.e., root or admin; else 0
1. Is_guest_login: 1 if the login is a guest, 0 otherwise
#### TIME RELATED TRAFFIC FEATURES OF EACH NETWORK CONNECTION VECTOR
1. Count: Number of connections to the same destination host as the current connection in the past two seconds
1. Srv_count: Number of connections to the same service (port number) as the current connection in th e past two seconds
1. Serror_rate: The percentage of connections that have activated the flag (4) s0, s1, s2 or s3, among the connections aggregated in count (23)
1. Srv_serror_rate: The percentage of connections that have activated the flag (4) s0, s1, s2 or s3, among the connections aggregated in srv_count (24)
1. Rerror_rate: The percentage of connections that have activated the flag (4) REJ, among the connections aggregated in count (23)
1. Srv_rerror_rate: The percentage of connections that have activated the flag (4) REJ, among the connections aggregated in srv_count (24)
1. Same_srv_rate: The percentage of connections that were to the same service, among the connections aggregated in count (23)
1. Diff_srv_rate: The percentage of connections that were to different services, among the connections aggregated in count (23)
1. Srv_diff_host_ rate: The percentage of connections that were to different destination machines among the connections aggregated in srv_count (24)
#### HOST BASED TRAFFIC FEATURES IN A NETWORK CONNECTION VECTOR
1. Dst_host_count: Number of connections having the same destination host IP address
1. Dst_host_srv_ count: Number of connections having the same port number
1. Dst_host_same srv_rate: The percentage of connections that were to the same service, among the connections aggregated in dst_host_count (32) 
1. Dst_host_diff srv_rate: The percentage of connections that were to different services, among the connections aggregated in dst_host_count (32)
1. Dst_host_same src_port_rate: The percentage of connections that were to the same source port, among the connections aggregated in dst_host_srv_c ount (33) 
1. Dst_host_srv diff_host_rate: The percentage of connections that were to different destination machines, among the connections aggregated in dst_host_srv_count (33)
1. Dst_host_serro r_rate: The percentage of connections that have activated the flag (4) s0, s1, s2 or s3, among the connections aggregated in dst_host_count (32)
1. Dst_host_srv_s error_rate: The percent of connections that have activated the flag (4) s0, s1, s2 or s3, among the connections aggregated in dst_host_srv_c ount (33)
1. Dst_host_rerro r_rate: The percentage of connections that have activated the flag (4) REJ, among the connections aggregated in dst_host_count (32)
1. Dst_host_srv_r error_rate: The percentage of connections that have activated the flag (4) REJ, among the connections aggregated in dst_host_srv_c ount (33)

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.describe(include='object')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.countplot(x=train["class"], palette=("Pastel2"))
abs_values = train["class"].value_counts(ascending=False).values
ax.bar_label(container=ax.containers[0], labels=abs_values)
ax.set_xticklabels(['normal', 'anomaly'])

plt.xlabel(None)
plt.show()

In [ ]:
test.info()

In [ ]:
test.head()

In [ ]:
test.describe(include='object')

In [ ]:
# remove missing values (NaN)
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
# duplicates
print(f"Number of duplicate rows in train: {train.duplicated().sum()}")
print(f"Number of duplicate rows in test: {test.duplicated().sum()}")

### Label encoding
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoding(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

label_encoding(train)
label_encoding(test)

train

### Scale and split data

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(['class'], axis=1)
# X_train = pd.concat([X_train, test], axis=0)
y = train['class']

Let's use [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html) since it's not so sensitive to outliers like StandardScaler and MinMaxScaler.

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_rem, y_train, y_rem = train_test_split(X, y, random_state=42, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, random_state=42, test_size=0.5)

## Supervised models

## Unsupervised models

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def print_stats(predictions, X, y, title):
    # Predict the anomalies
    print(len(predictions[predictions == -1]), " outliers from ", len(predictions))
    print("0 is anomaly, 1 is normal")
    predictions = [0 if p == -1 else 1 for p in predictions]
    print(classification_report(y, predictions))

    conf = confusion_matrix(y_pred=predictions,y_true=y)
    plt.title(title)
    ax= plt.subplot()
    sns.heatmap(conf, annot=True, fmt="", linewidths=2, cmap="Greens")
    ax.set_xlabel('Predicted');
    ax.set_ylabel('Real');
    ax.xaxis.set_ticklabels(['Normal', 'Anomaly']); 
    ax.yaxis.set_ticklabels(['Normal', 'Anomaly']);
    plt.show()

### One-class SVM

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score

# Hyperparameters grid
kernel_range = ['linear', 'poly', 'rbf', 'sigmoid']
nu_range = [0.1, 0.2, 0.3, 0.4, 0.5]
gamma_range = ['scale', 'auto']
degree_range = [2, 3, 4, 5]

# Store results
best_score = -1
best_params = {}

for kernel in kernel_range:
    for nu in nu_range:
        for gamma in gamma_range:
            for degree in degree_range:
                model = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma, degree=degree).fit(X_train)
                predictions = model.predict(X_train)
                predictions = [0 if p == -1 else 1 for p in predictions]

                score = accuracy_score(y_true=y_train, y_pred=predictions)
                if best_score < score:
                    best_score = score
                    best_params = {'kernel': kernel, 'nu': nu, 'gamma': gamma, 'degree': degree}

print("Best score:", best_score)
print("Best hyperparameters:", best_params) 

In [ ]:
from sklearn.svm import OneClassSVM

predictions = OneClassSVM().fit_predict(X_train)

In [ ]:
print_stats(predictions, X_train, y_train, "One-class SVM")

### Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import accuracy_score

# Hyperparameters grid
n_neighbors_range = range(5, 50)
algorithm_range = ['auto', 'ball_tree', 'kd_tree', 'brute']
leaf_size_range = range(20, 50)

# Store results
best_score = -1
best_params = {}

for n_neighbors in n_neighbors_range:
    for leaf_size in leaf_size_range:
        for algorithm in algorithm_range:
            model = LocalOutlierFactor(n_neighbors=n_neighbors, algorithm=algorithm, leaf_size=leaf_size).fit(X_train)
            predictions = model.predict(X_train)
            predictions = [0 if p == -1 else 1 for p in predictions]

            score = accuracy_score(y_true=y_train, y_pred=predictions)
            if best_score < score:
                best_score = score
                best_params = {'n_neighbors': n_neighbors, 'algorithm': algorithm, 'leaf_size': leaf_size}

print("Best score:", best_score)
print("Best hyperparameters:", best_params) 

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

predictions = LocalOutlierFactor().fit_predict(X_train)

In [ ]:
print_stats(predictions, X_train, y_train, "Local Outlier Factor")

### Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score

# Hyperparameters grid
n_estimators_range = range(100, 500, 5)
max_features_range = range(1, 42)

# Store results
best_score = -1
best_params = {}

for n_estimators in n_estimators_range:
    for max_features in max_features_range:
        model = IsolationForest(n_estimators=n_estimators, max_features=max_features).fit(X_train)
        predictions = model.predict(X_train)
        predictions = [0 if p == -1 else 1 for p in predictions]

        score = accuracy_score(y_true=y_train, y_pred=predictions)
        if best_score < score:
            best_score = score
            best_params = {'n_estimators': n_estimators, 'max_features': max_features}

print("Best score:", best_score)
print("Best hyperparameters:", best_params) 

In [ ]:
from sklearn.ensemble import IsolationForest

predictions = IsolationForest().fit_predict(X_train)

In [ ]:
print_stats(predictions, X_train, y_train, "Isolation Forest")

### Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score

# Hyperparameters grid
n_components_range = [1, 2, 3, 4, 5]
covariance_type_range = ['full', 'tied', 'diag', 'spherical']
n_init_range = [1, 2, 3, 4, 5]

# Store results
best_score = -1
best_params = {}

for n_components in n_components_range:
    for covariance_type in covariance_type_range:
        for n_init in n_init_range:
            gmm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, n_init=n_init).fit(X_train)
            predictions = gmm.predict(X_train)
            predictions = [0 if p == -1 else 1 for p in predictions]

            score = accuracy_score(y_true=y_train, y_pred=predictions)
            if best_score < score:
                best_score = score
                best_params = {'n_components': n_components, 'covariance_type': covariance_type, 'n_init': n_init}

print("Best score:", best_score)
print("Best hyperparameters:", best_params)        

In [ ]:
predictions = GaussianMixture().fit_predict(X_train)
print_stats(predictions, X_train, y_train, "Gaussian Mixture Model")

## Autoencoder
https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798